# 3.1 Consultas contra la base de datos local "ucm_nosql_pl"

## a.El Total de locales y terrazas por distrito y barrio: construye una consulta que permita obtener el total de locales y terrazas agrupados por cada distrito y barrio.


In [3]:
#------------Dependencies-----------#
import pandas as pd
import pymongo as mongo

In [9]:
def mostrar_resultado(result, n=5):
    """
    Convierte un cursor de MongoDB a DataFrame y muestra las primeras filas.

    Parámetros:
    - result: cursor devuelto por find() o aggregate()
    - n: número de filas a mostrar (por defecto 5)
    """
    df_resultado = pd.DataFrame(list(result))
    return df_resultado.head(n)


In [10]:
# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = mongo.MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$lookup': {
            'from': 'terrazas',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'terrazas_asociadas'
        }
    }, {
        '$group': {
            '_id': {
                'distrito': '$desc_distrito_local',
                'barrio': '$desc_barrio_local'
            },
            'total_locales': {
                '$sum': 1
            },
            'locales_con_terraza': {
                '$sum': {
                    '$cond': [
                        {
                            '$gt': [
                                {
                                    '$size': '$terrazas_asociadas'
                                }, 0
                            ]
                        }, 1, 0
                    ]
                }
            }
        }
    }
])

mostrar_resultado(result,5)

,_id,total_locales,locales_con_terraza
0,"{'distrito': 'moncloa-aravaca', 'barrio': 'ciu...",898,22
1,"{'distrito': 'villa de vallecas', 'barrio': 'c...",3667,64
2,"{'distrito': 'ciudad lineal', 'barrio': 'ventas'}",2790,54
3,"{'distrito': 'moratalaz', 'barrio': 'fontarron'}",376,17
4,"{'distrito': 'arganzuela', 'barrio': 'legazpi'}",508,49


## b.Tipos de licencias y cantidad de licencias por cada tipo: crea una consulta para contar cuántas licencias hay de cada tipo en los datos.

In [13]:
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['licencias'].aggregate([
    {
        '$group': {
            '_id': {
                'id_tipo_licencia': '$id_tipo_licencia'
            },
            'total_licencias_x_tipo': {
                '$sum': 1
            }
        }
    }
])
mostrar_resultado(result,5)

,_id,total_licencias_x_tipo
0,{'id_tipo_licencia': 4},57028
1,{'id_tipo_licencia': 0},5956
2,{'id_tipo_licencia': 3},5813
3,{'id_tipo_licencia': 1},51249
4,{'id_tipo_licencia': 2},30783


## c.Listado de locales y terrazas con licencias “En trámite”: diseña una consulta que filtre y devuelva un listado detallado de locales y terrazas cuyo estado de licencia sea “En trámite”.

In [ ]:
 # Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$lookup': {
            'from': 'terrazas',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'terrazas_asociadas'
        }
    }, {
        '$lookup': {
            'from': 'licencias',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'licencias_x_local'
        }
    }, {
        '$match': {
            'licencias_x_local.desc_tipo_situacion_licencia': 'en tramitación'
        }
    }, {
        '$project': {
            'id_local': 1,
            'desc_distrito_local': 1,
            'desc_barrio_local': 1,
            'licencias_en_tramite': {
                '$filter': {
                    'input': '$licencias_x_local',
                    'as': 'lic',
                    'cond': {
                        '$eq': [
                            '$$lic.desc_tipo_situacion_licencia', 'en tramitación'
                        ]
                    }
                }
            }
        }
    }
])
mostrar_resultado(result,5)

![MongoDB_Carga_Exitosa](imagenes_capturas_mongo/query_C.png)


**Sin indices creados, las querys pueden ser muy largas si se trata de hacer uniones por referencia**

## d.Consulta por sección, división y epígrafe de la actividad comercial: crea una consulta para clasificar locales y terrazas según los campos sección, división y epígrafe.

In [14]:
# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$lookup': {
            'from': 'terrazas',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'terrazas_x_locales'
        }
    }, {
        '$match': {
            'actividades.0': {
                '$exists': True
            },
            'terrazas_x_locales.0': {
                '$exists': True
            }
        }
    }, {
        '$unwind': {
            'path': '$actividades'
        }
    }, {
        '$unwind': {
            'path': '$terrazas_x_locales'
        }
    }, {
        '$match': {
            '$and': [
                {
                    'actividades.desc_seccion': {
                        '$ne': None
                    }
                }, {
                    'actividades.desc_division': {
                        '$ne': None
                    }
                }, {
                    'actividades.desc_epigrafe': {
                        '$ne': None
                    }
                }
            ]
        }
    }, {
        '$group': {
            '_id': {
                'seccion': '$actividades.desc_seccion',
                'division': '$actividades.desc_division',
                'epigrafe': '$actividades.desc_epigrafe'
            },
            'total_locales_con_terraza': {
                '$sum': 1
            }
        }
    }
])
mostrar_resultado(result,5) #MUY PESADO SIN INDICES

,_id,total_locales_con_terraza
0,"{'seccion': 'actividades artísticas, recreativ...",2
1,"{'seccion': 'educación', 'division': 'educació...",1
2,"{'seccion': 'actividades artísticas, recreativ...",7
3,{'seccion': 'comercio al por mayor y al por me...,1
4,"{'seccion': 'hostelería', 'division': 'servici...",33


## e.Actividad económica más frecuente por barrio y distrito: diseña una consulta que identifique cuál es la actividad económica predominante en cada barrio y distrito.

In [15]:
# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$unwind': {
            'path': '$actividades'
        }
    }, {
        '$group': {
            '_id': {
                'actividad': '$actividades.desc_division',
                'distrito': '$desc_distrito_local',
                'barrio': '$desc_barrio_local'
            },
            'numero_de_actividades': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            '_id.distrito': 1,
            '_id.barrio': 1,
            'numero_de_actividades': -1
        }
    }, {
        '$group': {
            '_id': {
                'distrito': '$_id.distrito',
                'barrio': '$_id.barrio'
            },
            'actividad_mas_frecuente': {
                '$first': '$_id.actividad'
            },
            'frecuencia': {
                '$first': '$numero_de_actividades'
            }
        }
    }
])
mostrar_resultado(result,5)

,_id,actividad_mas_frecuente,frecuencia
0,"{'distrito': 'carabanchel', 'barrio': 'san isi...",valor nulo en origen,888
1,"{'distrito': 'fuencarral-el pardo', 'barrio': ...",valor nulo en origen,6
2,"{'distrito': 'retiro', 'barrio': 'los jeronimos'}",servicios de comidas y bebidas,100
3,"{'distrito': 'arganzuela', 'barrio': 'atocha'}","comercio al por menor, excepto de vehículos de...",81
4,"{'distrito': 'centro', 'barrio': 'cortes'}",servicios de comidas y bebidas,448


## f.Actualización de horarios de apertura y cierre de ciertos locales: modifica los horarios de apertura y cierre de un conjunto seleccionado de locales según un criterio que tú determines.

![MongoDB_Carga_Exitosa](imagenes_capturas_mongo/Query_ultima.png)

##  Actualización de horarios de cierre en zonas turísticas

Como parte del mantenimiento y mejora del modelo de datos, se ha realizado una **actualización de los horarios de cierre (`hora_cierre1` y `hora_cierre2`)** de ciertos locales ubicados en zonas históricas y turísticas de la ciudad.

###  Criterio utilizado
Se han seleccionado los locales cuya propiedad `desc_distrito_local` corresponde a los siguientes distritos:

- **Centro**
- **Chamberí**
- **Retiro**

Estos distritos representan áreas céntricas con alta densidad de turismo, ocio nocturno y residencias, donde suelen aplicarse **normativas municipales de control del ruido y del descanso vecinal**.

###  Modificación aplicada
Se ha actualizado el horario de cierre de estos locales a las **01:00:00** en ambos campos de hora:

```json
{
  "hora_cierre1": "01:00:00",
  "hora_cierre2": "01:00:00"
}


## 3.2 Creación y uso de indices

![MongoDB_Creacion_Indices](imagenes_capturas_mongo/creacion_indices.png)

**Creación de indices**

![MongoDB_Prueba_Indice](imagenes_capturas_mongo/indice_usado.png)
**Prueba de uso de indice**

## Plan de ejecución de las consultas y diferencia de tiempos

## a.El Total de locales y terrazas por distrito y barrio: construye una consulta que permita obtener el total de locales y terrazas agrupados por cada distrito y barrio.


In [17]:
# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = mongo.MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$lookup': {
            'from': 'terrazas',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'terrazas_asociadas'
        }
    }, {
        '$group': {
            '_id': {
                'distrito': '$desc_distrito_local',
                'barrio': '$desc_barrio_local'
            },
            'total_locales': {
                '$sum': 1
            },
            'locales_con_terraza': {
                '$sum': {
                    '$cond': [
                        {
                            '$gt': [
                                {
                                    '$size': '$terrazas_asociadas'
                                }, 0
                            ]
                        }, 1, 0
                    ]
                }
            }
        }
    }
])

mostrar_resultado(result,5)

,_id,total_locales,locales_con_terraza
0,"{'distrito': 'usera', 'barrio': 'almendrales'}",1058,19
1,"{'distrito': 'usera', 'barrio': 'moscardo'}",1492,40
2,"{'distrito': 'chamartin', 'barrio': 'castilla'}",756,43
3,"{'distrito': 'moncloa-aravaca', 'barrio': 'val...",101,2
4,"{'distrito': 'hortaleza', 'barrio': 'canillas'}",1964,61


## b.Tipos de licencias y cantidad de licencias por cada tipo: crea una consulta para contar cuántas licencias hay de cada tipo en los datos.

In [18]:
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['licencias'].aggregate([
    {
        '$group': {
            '_id': {
                'id_tipo_licencia': '$id_tipo_licencia'
            },
            'total_licencias_x_tipo': {
                '$sum': 1
            }
        }
    }
])
mostrar_resultado(result,5)

,_id,total_licencias_x_tipo
0,{'id_tipo_licencia': 3},5813
1,{'id_tipo_licencia': 1},51249
2,{'id_tipo_licencia': 2},30783
3,{'id_tipo_licencia': 4},57028
4,{'id_tipo_licencia': 0},5956


## c.Listado de locales y terrazas con licencias “En trámite”: diseña una consulta que filtre y devuelva un listado detallado de locales y terrazas cuyo estado de licencia sea “En trámite”.

## Creación de indices para mejorar el tiempo de respuesta de esta query que antes pasó mas de 5 minutos y no devolvía respuesta

![MongoDB_Prueba_Indice](imagenes_capturas_mongo/indice_usado.png)

In [21]:
 # Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$lookup': {
            'from': 'terrazas',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'terrazas_asociadas'
        }
    }, {
        '$lookup': {
            'from': 'licencias',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'licencias_x_local'
        }
    }, {
        '$match': {
            'licencias_x_local.desc_tipo_situacion_licencia': 'en tramitación'
        }
    }, {
        '$project': {
            'id_local': 1,
            'desc_distrito_local': 1,
            'desc_barrio_local': 1,
            'licencias_en_tramite': {
                '$filter': {
                    'input': '$licencias_x_local',
                    'as': 'lic',
                    'cond': {
                        '$eq': [
                            '$$lic.desc_tipo_situacion_licencia', 'en tramitación'
                        ]
                    }
                }
            }
        }
    }
])
mostrar_resultado(result,5)

,_id,id_local,desc_distrito_local,desc_barrio_local,licencias_en_tramite
0,685a81586586723066b49382,20000596,arganzuela,chopera,"[{'_id': 685a815d6586723066b87c64, 'id_local':..."
1,685a81586586723066b49385,20000709,arganzuela,acacias,"[{'_id': 685a815d6586723066b88458, 'id_local':..."
2,685a81586586723066b49387,20000729,arganzuela,delicias,"[{'_id': 685a815d6586723066b87da0, 'id_local':..."
3,685a81586586723066b49388,20000756,arganzuela,palos de la frontera,"[{'_id': 685a815d6586723066b881bf, 'id_local':..."
4,685a81586586723066b4938a,20000764,arganzuela,acacias,"[{'_id': 685a815d6586723066b88492, 'id_local':..."


## HA TARDADO SOLO 3 S 592 MS IMPRESIONANTE¡

## d.Consulta por sección, división y epígrafe de la actividad comercial: crea una consulta para clasificar locales y terrazas según los campos sección, división y epígrafe.

In [22]:
# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$unwind': {
            'path': '$actividades'
        }
    }, {
        '$group': {
            '_id': {
                'actividad': '$actividades.desc_division',
                'distrito': '$desc_distrito_local',
                'barrio': '$desc_barrio_local'
            },
            'numero_de_actividades': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            '_id.distrito': 1,
            '_id.barrio': 1,
            'numero_de_actividades': -1
        }
    }, {
        '$group': {
            '_id': {
                'distrito': '$_id.distrito',
                'barrio': '$_id.barrio'
            },
            'actividad_mas_frecuente': {
                '$first': '$_id.actividad'
            },
            'frecuencia': {
                '$first': '$numero_de_actividades'
            }
        }
    }
])
mostrar_resultado(result,5)


,_id,actividad_mas_frecuente,frecuencia
0,"{'distrito': 'ciudad lineal', 'barrio': 'colina'}",valor nulo en origen,80
1,"{'distrito': 'puente de vallecas', 'barrio': '...",valor nulo en origen,675
2,"{'distrito': 'latina', 'barrio': 'campamento'}",valor nulo en origen,283
3,"{'distrito': 'chamartin', 'barrio': 'castilla'}",valor nulo en origen,210
4,"{'distrito': 'hortaleza', 'barrio': 'canillas'}",valor nulo en origen,632


# 4. MODELO DE DATOS (VERSION 2): EXTENSION CON ALOJAMIENTOS TURISTICOS

## 4.1 INCORPORACION DE DATOS DE ALOJAMIENTOS TURISTICOS

**A.Revisión y obtención del dataset**

In [33]:
# Cargar el archivo CSV (ajusta la ruta a la tuya si es necesario)
ruta_csv = "data_files/airbnb_alojamientos.csv"
df_airbnb = pd.read_csv(
     ruta_csv,
     sep = ",",
     skip_blank_lines = True,
     on_bad_lines ='skip',
     encoding = 'utf-8'
    )


**Revisión del Data Frame y depuración de su contenido**

In [34]:
df_airbnb.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,21853,Bright and airy room,83531,Abdel,Latina,Cármenes,40.40381,-3.74130,Private room,29.0,4,33,2018-07-15,0.26,2,233,0,NaN
1,30320,Great Vacational Apartments,130907,Dana,Centro,Sol,40.41476,-3.70418,Entire home/apt,NaN,5,172,2022-09-26,0.96,3,0,0,NaN
2,30959,Beautiful loft in Madrid Center,132883,Angela,Centro,Embajadores,40.41259,-3.70105,Entire home/apt,NaN,3,8,2017-05-30,0.07,1,0,0,NaN
3,40916,Holiday Apartment Madrid Center,130907,Dana,Centro,Universidad,40.42247,-3.70577,Entire home/apt,NaN,5,49,2021-12-11,0.28,3,0,0,NaN
4,62423,MAGIC ARTISTIC HOUSE IN THE CENTER OF MADRID,303845,Arturo,Centro,Justicia,40.41884,-3.69655,Private room,77.0,1,227,2025-02-20,2.73,3,298,46,NaN


In [35]:
df_airbnb.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
count,2.528800e+04,2.528800e+04,25288.000000,25288.000000,19274.000000,25288.000000,25288.000000,20091.000000,25288.000000,25288.000000,25288.000000
mean,6.439293e+17,2.583530e+08,40.421572,-3.693851,138.965082,8.828061,47.688508,1.778151,36.220263,137.977381,14.286816
std,5.357949e+17,2.083118e+08,0.023441,0.027972,433.623184,31.799013,89.272627,1.914170,81.239399,132.862637,21.846090
min,2.185300e+04,7.952000e+03,40.331400,-3.883990,8.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000
25%,3.676095e+07,5.168513e+07,40.409288,-3.707420,65.000000,1.000000,1.000000,0.360000,1.000000,0.000000,0.000000
50%,8.115619e+17,2.228602e+08,40.420382,-3.700932,97.000000,2.000000,10.000000,1.130000,3.000000,96.000000,4.000000
75%,1.142574e+18,4.488906e+08,40.431617,-3.684771,142.000000,4.000000,53.000000,2.630000,19.000000,268.000000,21.000000
max,1.369179e+18,6.821759e+08,40.577290,-3.545904,23124.000000,1125.000000,1080.000000,28.570000,341.000000,365.000000,263.000000


**Respecto al contenido del DATASET el original CSV contiene 25381 filas y el DF contiene 25288 filas. Por lo que han habido menos del 1% de los datos, aún sería bueno a futuro evaluar ese porcentaje si tiene algun dato fuera de lugar que diera error etc para tener el maximo contenido posible**

In [39]:
print(df_airbnb.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25288 entries, 0 to 25287
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              25288 non-null  int64  
 1   name                            25288 non-null  object 
 2   host_id                         25288 non-null  int64  
 3   host_name                       25269 non-null  object 
 4   neighbourhood_group             25288 non-null  object 
 5   neighbourhood                   25288 non-null  object 
 6   latitude                        25288 non-null  float64
 7   longitude                       25288 non-null  float64
 8   room_type                       25288 non-null  object 
 9   price                           19274 non-null  float64
 10  minimum_nights                  25288 non-null  int64  
 11  number_of_reviews               25288 non-null  int64  
 12  last_review                     

In [41]:
print(df_airbnb.isnull().sum())

id                                    0
name                                  0
host_id                               0
host_name                            19
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                              6014
minimum_nights                        0
number_of_reviews                     0
last_review                        5197
reviews_per_month                  5197
calculated_host_listings_count        0
availability_365                      0
number_of_reviews_ltm                 0
license                           22310
dtype: int64


**Se va a proceder a rellenar los campos con valores nulos, con datos acorde al tipo de dato para que no se quede en un NULL al momento de importarlo a la base de DATOSNOSQL**

In [ ]:
# Rellenar valores nulos por tipo de dato y significado
df_airbnb["host_name"].fillna("no_data", inplace=True)
df_airbnb["license"].fillna("no_license", inplace=True)
df_airbnb["last_review"].fillna("no_review_date", inplace=True)
df_airbnb["reviews_per_month"].fillna(0.0, inplace=True)
df_airbnb["price"].fillna(0.0, inplace=True)


In [44]:
print(df_airbnb.isnull().sum())

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
number_of_reviews_ltm             0
license                           0
dtype: int64


In [45]:
#Normalizar nombres de las columnas
def limpiar_df(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    df = df.map(lambda x: x.strip().lower() if isinstance(x, str) else x)
    #df = df.drop_duplicates() -- Si a futuro hiciera falta elimminar duplicados de algun data frame.
    return df

df_airbnb = limpiar_df(df_airbnb)

In [46]:
df_airbnb.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,21853,bright and airy room,83531,abdel,latina,cármenes,40.40381,-3.74130,private room,29.0,4,33,2018-07-15,0.26,2,233,0,no_license
1,30320,great vacational apartments,130907,dana,centro,sol,40.41476,-3.70418,entire home/apt,0.0,5,172,2022-09-26,0.96,3,0,0,no_license
2,30959,beautiful loft in madrid center,132883,angela,centro,embajadores,40.41259,-3.70105,entire home/apt,0.0,3,8,2017-05-30,0.07,1,0,0,no_license
3,40916,holiday apartment madrid center,130907,dana,centro,universidad,40.42247,-3.70577,entire home/apt,0.0,5,49,2021-12-11,0.28,3,0,0,no_license
4,62423,magic artistic house in the center of madrid,303845,arturo,centro,justicia,40.41884,-3.69655,private room,77.0,1,227,2025-02-20,2.73,3,298,46,no_license


In [48]:
df_airbnb.rename(columns={
    "neighbourhood_group": "desc_distrito_local",
    "neighbourhood": "desc_barrio_local"
}, inplace=True)


In [49]:
df_airbnb.head()

,id,name,host_id,host_name,desc_distrito_local,desc_barrio_local,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,21853,bright and airy room,83531,abdel,latina,cármenes,40.40381,-3.74130,private room,29.0,4,33,2018-07-15,0.26,2,233,0,no_license
1,30320,great vacational apartments,130907,dana,centro,sol,40.41476,-3.70418,entire home/apt,0.0,5,172,2022-09-26,0.96,3,0,0,no_license
2,30959,beautiful loft in madrid center,132883,angela,centro,embajadores,40.41259,-3.70105,entire home/apt,0.0,3,8,2017-05-30,0.07,1,0,0,no_license
3,40916,holiday apartment madrid center,130907,dana,centro,universidad,40.42247,-3.70577,entire home/apt,0.0,5,49,2021-12-11,0.28,3,0,0,no_license
4,62423,magic artistic house in the center of madrid,303845,arturo,centro,justicia,40.41884,-3.69655,private room,77.0,1,227,2025-02-20,2.73,3,298,46,no_license


## SUBIDA DE DATOS A BBDD NOSQL VERSION 2

In [50]:
from pymongo import MongoClient

# Conexión con MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["ucm_nosql_pl"]
coleccion = db["alojamientos_turisticos"]

#Convertir el DataFrame a lista de documentos
documentos = df_airbnb.to_dict(orient="records")

#Insertar en MongoDB
resultado = coleccion.insert_many(documentos)

#Confirmación
print(f"Se insertaron {len(resultado.inserted_ids)} documentos en 'alojamientos_turisticos'")


Se insertaron 25288 documentos en 'alojamientos_turisticos'


### CREACION DE INDICES EN LOS CAMPOS QUE SE VAN A USAR PARA REFERENCIAS CON LAS DEMAS COLECCIONES

![MongoDB_Indices_Version2](imagenes_capturas_mongo/indices_creados_version2_airbnb.png)

### a.Propuesta de mejora del modelo: diseña modificaciones o ampliaciones en tu modelo de datos actual para incorporar la información de alojamientos turísticos. Justifica las decisiones tomadas, asegurándote de mantener un diseño eficiente para las consultas posteriores.

### 4.2 Mejora del modelo de datos: integración de alojamientos turísticos

Para mejorar el modelo actual, se ha incorporado una nueva colección denominada `alojamientos_turisticos` que contiene información detallada sobre los alojamientos tipo Airbnb en la ciudad de Madrid.

#### Decisión de diseño
Los alojamientos se han añadido como **colección independiente**, y no embebidos en locales o barrios, porque:

- No están ligados a un `id_local` concreto.
- Poseen características propias (precio, disponibilidad, anfitrión, tipo de alojamiento).
- Su volumen puede crecer de forma independiente del resto del modelo.

#### Relación con el modelo actual
La relación con locales y terrazas se establece mediante los campos:

- `desc_barrio_local`
- `desc_distrito_local`

Esta clave lógica permite realizar consultas conjuntas usando `$lookup`, `$group` o `$match` según el contexto.

#### Eficiencia en consultas
Para garantizar un diseño eficiente, se ha creado un **índice compuesto** en `alojamientos_turisticos` sobre `{ desc_distrito_local: 1, desc_barrio_local: 1 }`.
Esto optimiza búsquedas geográficas y facilita agregaciones cruzadas con otras colecciones.

> Este diseño modular favorece la escalabilidad del sistema y mantiene la claridad de las relaciones entre las entidades, sin comprometer el rendimiento.
